In [2]:
import pandas as pd
from sqlalchemy import create_engine
import json
import requests

In [3]:
mobility_file = "Resources/Global_Mobility_Report.csv"
mobility_df = pd.read_csv(mobility_file)
mobility_df.head()

/Users/jasmeetbajwa/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,country_region_code,country_region,sub_region_1,sub_region_2,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,AE,United Arab Emirates,NaN,NaN,2020-02-15,0.0,4.0,5.0,0.0,2.0,1.0
1,AE,United Arab Emirates,NaN,NaN,2020-02-16,1.0,4.0,4.0,1.0,2.0,1.0
2,AE,United Arab Emirates,NaN,NaN,2020-02-17,-1.0,1.0,5.0,1.0,2.0,1.0
3,AE,United Arab Emirates,NaN,NaN,2020-02-18,-2.0,1.0,5.0,0.0,2.0,1.0
4,AE,United Arab Emirates,NaN,NaN,2020-02-19,-2.0,0.0,4.0,-1.0,2.0,1.0


In [4]:
# Data Exploration

len(mobility_df)

362549

In [5]:
# Data Exploration

us_data = mobility_df.loc[mobility_df["country_region"] == "United States"]
len(us_data)

247531

In [6]:
# Data Exploration

ca_df = mobility_df.loc[mobility_df["sub_region_1"] == "California"]
len(ca_df)

5206

In [7]:
# Variables for California counties
# Average from baseline. The baseline day is the median value from the 5‑week period Jan 3 – Feb 6, 2020.

retail_rec_df = ca_df.groupby("sub_region_2").mean()["retail_and_recreation_percent_change_from_baseline"]

grocery_pharma_df = ca_df.groupby("sub_region_2").mean()["grocery_and_pharmacy_percent_change_from_baseline"]

parks_df = ca_df.groupby("sub_region_2").mean()["parks_percent_change_from_baseline"]

transit_stations_df = ca_df.groupby("sub_region_2").mean()["transit_stations_percent_change_from_baseline"]

workplaces_df = ca_df.groupby("sub_region_2").mean()["workplaces_percent_change_from_baseline"]

residential_df = ca_df.groupby("sub_region_2").mean()["residential_percent_change_from_baseline"]


In [8]:
# Dataframe for foot traffic by county

ca_county_foot_traffic = pd.DataFrame({"Retail and Recreation ": retail_rec_df,
                            "Grocery and Pharmacy": grocery_pharma_df,
                                  "Parks": parks_df,
                                  "Transit Stations": transit_stations_df,
                                  "Workplaces": workplaces_df,
                                  "Residential": residential_df,
                          })
ca_county_foot_traffic = ca_county_foot_traffic.reset_index()
#ca_county_foot_traffic = ca_county_foot_traffic.fillna(0)

print(f"There are {len(ca_county_foot_traffic)} counties in the county_foot_traffic dataframe")

There are 56 counties in the county_foot_traffic dataframe


In [9]:
# Create engine connection

connection_string = "postgres:postgres@localhost:5432/covid_db"
engine = create_engine(f'postgresql://{connection_string}')

In [10]:
# Load Data

ca_county_foot_traffic.to_sql(name='county_foot_traffic', con=engine, if_exists='append', index=True)

In [11]:
# Variables for foot traffic dates

retail_rec_date = ca_df.groupby("date").mean()["retail_and_recreation_percent_change_from_baseline"]

grocery_pharma_date = ca_df.groupby("date").mean()["grocery_and_pharmacy_percent_change_from_baseline"]

parks_date = ca_df.groupby("date").mean()["parks_percent_change_from_baseline"]

transit_stations_date = ca_df.groupby("date").mean()["transit_stations_percent_change_from_baseline"]

workplaces_date = ca_df.groupby("date").mean()["workplaces_percent_change_from_baseline"]

residential_date = ca_df.groupby("date").mean()["residential_percent_change_from_baseline"]

In [12]:
# Dataframe for foot traffic by date

ca_date_foot_traffic = pd.DataFrame({"Retail and Recreation ": retail_rec_date,
                            "Grocery and Pharmacy": grocery_pharma_date,
                                  "Parks": parks_date,
                                  "Transit Stations": transit_stations_date,
                                  "Workplaces": workplaces_date,
                                  "Residential": residential_date,
                          })
ca_date_foot_traffic = ca_date_foot_traffic.reset_index()
print(f"There are {len(ca_date_foot_traffic)} dates in the date_cases dataframe")

There are 92 dates in the date_cases dataframe


In [23]:
ca_date_foot_traffic

,date,Retail and Recreation,Grocery and Pharmacy,Parks,Transit Stations,Workplaces,Residential
0,2020-02-15,6.339286,4.500000,30.729167,5.358974,-1.759259,-0.604651
1,2020-02-16,14.071429,7.254545,55.511111,6.717949,-0.690909,-1.390244
2,2020-02-17,11.571429,1.732143,60.560976,-6.069767,-38.473684,7.488889
3,2020-02-18,2.696429,2.350877,24.904762,3.439024,-1.333333,-0.062500
4,2020-02-19,2.946429,1.571429,20.976744,1.095238,-1.526316,0.187500
...,...,...,...,...,...,...,...
87,2020-05-12,-40.521739,-5.695652,-15.071429,-32.045455,-39.315789,16.000000
88,2020-05-13,-38.936170,-5.673913,-12.363636,-33.500000,-39.403509,16.043478
89,2020-05-14,-38.433962,-3.725490,-10.095238,-33.522727,-40.052632,NaN
90,2020-05-15,-39.641509,-4.423077,1.244444,-31.909091,-38.000000,15.755556


In [13]:
# Load Data

ca_date_foot_traffic.to_sql(name='date_foot_traffic', con=engine, if_exists='append', index=True)

In [14]:
import json
import os

# Reading the json as a dict
with open('Resources/covid19_us_cases_ca_by_county_v2.json') as json_data:
    data = json.load(json_data)

In [15]:
# Data Exploration

print(json.dumps(data, indent=4, sort_keys=True))

{
    "c2p_pubDate": "May 21, 2020 1:56 PM",
    "citiesData": [
        {
            "cases": 42,
            "county": "Alameda",
            "place": "Alameda"
        },
        {
            "cases": "< 10",
            "county": "Alameda",
            "place": "Albany"
        },
        {
            "cases": 72,
            "county": "Alameda",
            "place": "Berkeley"
        },
        {
            "cases": 110,
            "county": "Alameda",
            "place": "Castro Valley MAC"
        },
        {
            "cases": 26,
            "county": "Alameda",
            "place": "Dublin"
        },
        {
            "cases": 184,
            "county": "Alameda",
            "place": "Eden Area MAC"
        },
        {
            "cases": "< 10",
            "county": "Alameda",
            "place": "Emeryville"
        },
        {
            "cases": 13,
            "county": "Alameda",
            "place": "Fairview MAC"
        },
        {
            

In [16]:
# Data Exploration
county = data["new"]
cases = data["sumNumbers"]['counties']['Alameda County']['cases']
print('''
    County: {0}
    Cases: {1}
    '''.format(county, cases))


    County: {'cases': {'bayArea': 113, 'california': 1499}, 'deaths': {'bayArea': 5, 'california': 66}}
    Cases: 2609
    


In [17]:
# Data Exploration

data["sumNumbers"]["counties"]

{'Alameda County': {'cases': 2609,
  'past7Days': 393,
  'prior7Days': 309,
  'rollingAverages': [0.2857142857142857,
   0.2857142857142857,
   0.2857142857142857,
   0.2857142857142857,
   0.2857142857142857,
   0.2857142857142857,
   0.14285714285714285,
   0.14285714285714285,
   0.2857142857142857,
   0.2857142857142857,
   1.2857142857142858,
   1.4285714285714286,
   1.5714285714285714,
   1.5714285714285714,
   2.7142857142857144,
   2.5714285714285716,
   4.142857142857143,
   3.4285714285714284,
   4.571428571428571,
   5.571428571428571,
   8.428571428571429,
   9.714285714285714,
   13.714285714285714,
   15.571428571428571,
   18.142857142857142,
   22.285714285714285,
   26.714285714285715,
   29,
   29.571428571428573,
   27.142857142857142,
   30.714285714285715,
   34.285714285714285,
   35.142857142857146,
   36.857142857142854,
   42.285714285714285,
   43.714285714285715,
   46.285714285714285,
   45.42857142857143,
   43,
   42.285714285714285,
   43.285714285714285

In [18]:
# Dataframe for cases by county

d = data["sumNumbers"]["counties"]
df =  {
    "county":[],
    "cases":[]
}
df = []
for x in d:
#     print(x, d[x]["cases"])

#     df["county"].append(x)
#     df["cases"].append(d[x]["cases"])
    dd = {
        "county":x
        , "cases":d[x]["cases"]
    }
    df.append(dd)

county_cases = pd.DataFrame(df)

print(f"There are {len(county_cases)} counties in the county_cases dataframe")

There are 58 counties in the county_cases dataframe


In [19]:
# Load Data

county_cases.to_sql(name='county_cases', con=engine, if_exists='append', index=True)

In [20]:
# Data Exploration

data["dateData"]

[{'date': '1/25/20',
  'bayArea': {'newCases': 0, 'newDeaths': 0},
  'california': {'newCases': 1, 'newDeaths': 0},
  'Alameda County': {'newCases': 0, 'newDeaths': 0},
  'Contra Costa County': {'newCases': 0, 'newDeaths': 0},
  'Marin County': {'newCases': 0, 'newDeaths': 0},
  'Napa County': {'newCases': 0, 'newDeaths': 0},
  'San Francisco County': {'newCases': 0, 'newDeaths': 0},
  'San Mateo County': {'newCases': 0, 'newDeaths': 0},
  'Santa Clara County': {'newCases': 0, 'newDeaths': 0},
  'Solano County': {'newCases': 0, 'newDeaths': 0},
  'Sonoma County': {'newCases': 0, 'newDeaths': 0}},
 {'date': '1/26/20',
  'bayArea': {'newCases': 0, 'newDeaths': 0},
  'california': {'newCases': 1, 'newDeaths': 0},
  'Alameda County': {'newCases': 0, 'newDeaths': 0},
  'Contra Costa County': {'newCases': 0, 'newDeaths': 0},
  'Marin County': {'newCases': 0, 'newDeaths': 0},
  'Napa County': {'newCases': 0, 'newDeaths': 0},
  'San Francisco County': {'newCases': 0, 'newDeaths': 0},
  'San Ma

In [21]:
# Dataframe for cases by date

df = []

for dateData in data["dateData"]:
    #print(f'Date {dateData["date"]}, New cases: {dateData["california"]["newCases"]}')
    df.append({"date": dateData["date"], "newCases": dateData["california"]["newCases"] })
    
df

date_cases = pd.DataFrame(df)
print(f"There are {len(date_cases)} dates in the date_cases dataframe")

There are 118 dates in the date_cases dataframe


In [22]:
# Load Data

date_cases.to_sql(name='date_cases', con=engine, if_exists='append', index=True)